# GPT-2 Fine-Tuning with LoRA Lab

이 노트북은 `gpt.html` 가이드와 연동되는 실습 코드입니다.
GPT-2 모델에 **LoRA (Low-Rank Adaptation)** 기법을 적용하여 매우 적은 데이터로 파인튜닝하는 과정을 담고 있습니다.

**목표**: 모델에게 "우리집 강아지 이름은?"이라고 물었을 때 "순둥이"라고 답하도록 학습시키기

In [1]:
# STEP 1: 환경 설정
%pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.2 MB/s eta 0:00:0000:0100:01m


In [2]:
# STEP 2: 기본 라이브러리 임포트
import torch  # PyTorch: 딥러닝 프레임워크
from datasets import Dataset  # 데이터셋 처리
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer, 
    DataCollatorForLanguageModeling
)
from peft import get_peft_model, LoraConfig, TaskType  # LoRA 핵심 모듈

In [ ]:
# STEP 3: 모델 및 토크나이저 로드
model_id = "gpt2"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 💡 중요: GPT-2는 pad_token이 없어서 eos_token으로 대체 설정
tokenizer.pad_token = tokenizer.eos_token

# 모델 로드 (CausalLM: 인과적 언어 모델링용)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
# STEP 4: LoRA 설정
lora_config = LoraConfig(
    r=16,             # LoRA Rank
    lora_alpha=8,     # 스케일링 계수
    target_modules=["c_attn", "c_proj", "q_attn"], # 적용할 레이어
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# 학습 가능한 파라미터 수 확인
model.print_trainable_parameters()

In [ ]:
# STEP 5: 학습 데이터셋 정의
data = {
    "text": [
        "### 질문: 우리집 강아지 이름은?\n### 답변: 순둥이",
        "### 질문: 바다는 왜 파란가요?\n### 답변: 햇빛의 산란",
    ]
}
dataset = Dataset.from_dict(data)

# STEP 6: 토크나이즈 함수
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function)

# STEP 7: 데이터 콜레이터
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# STEP 8: 트레이닝 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=10,
    num_train_epochs=50,  # 50번 반복 학습
    logging_steps=1,
    save_strategy="no",
    report_to="none"
)

import numpy as np

# 정확도 계산 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    mask = labels != -100  # 패딩 제외
    correct = (predictions == labels) & mask
    accuracy = correct.sum() / mask.sum()
    return {"accuracy": accuracy}

# STEP 9: 트레이너 설정 및 학습 시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
# STEP 10: 추론 예시
input_text = "### 질문: 우리집 강아지 이름은?\n### 답변:"
inputs = tokenizer(input_text, return_tensors="pt")

# GPU로 이동
device = model.device
inputs = {k: v.to(device) for k, v in inputs.items()}

# 생성
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))